In [2]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-1gt75tsz
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-1gt75tsz
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=67ad6f4f656c41dd768ce21291a0fcf167bc7c59e13957f89654035e92d7b43d
  Stored in directory: /tmp/pip-ephem-wheel-cache-t2c1bftm/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [7]:
%%cu
#include<bits/stdc++.h>
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <random>

__global__ void nspdp_kernel(int* graph, int* distances, int num_nodes) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < num_nodes) {
        for (int i = 0; i < num_nodes; ++i) {
            if (i != tid && graph[i * num_nodes + tid] != INT_MAX) {
                atomicMin(&distances[i * num_nodes + tid], distances[tid * num_nodes + tid] + graph[i * num_nodes + tid]);
            }
        }
    }
}

void nspdp_sequential(std::vector<int>& graph, std::vector<int>& distances, int num_nodes) {
    for (int k = 0; k < num_nodes; ++k) {
        for (int i = 0; i < num_nodes; ++i) {
            for (int j = 0; j < num_nodes; ++j) {
                if (i != j && graph[i * num_nodes + k] != INT_MAX && graph[k * num_nodes + j] != INT_MAX) {
                    int new_distance = graph[i * num_nodes + k] + graph[k * num_nodes + j];
                    if (new_distance < graph[i * num_nodes + j]) {
                        distances[i * num_nodes + j] = new_distance;
                    }
                }
            }
        }
    }
}


void generate_random_graph(std::vector<int>& graph, int num_nodes) {
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dis(1, 10);  // Random weights between 1 and 10

    for (int i = 0; i < num_nodes; ++i) {
        for (int j = 0; j < num_nodes; ++j) {
            if (i == j) {
                graph[i * num_nodes + j] = 0;  // Distance from a node to itself is 0
            } else {
                graph[i * num_nodes + j] = dis(gen);  // Generate random weight
            }
        }
    }
}
void gg(std::vector<int>& graph,int n){
    for(int i=0;i<n;i++){
        for(int j=0;j<n;j++){
            std::cout<<graph[i*n+j]<<" ";
        }
        std::cout<<"\n";
    }
}
void dd(std::vector<int>& graph,int n){
    for(int i=0;i<n;i++){
        for(int j=0;j<n;j++){
            std::cout<<graph[i*n+j]<<" ";
        }
        std::cout<<"\n";
    }
}
int main() {
    int num_nodes=5;
    //std::cout << "Enter the number of nodes in the graph: ";
    //std::cin >> num_nodes;

    std::vector<int> graph(num_nodes * num_nodes);
    std::vector<int> distances(num_nodes * num_nodes, INT_MAX);

    generate_random_graph(graph, num_nodes);

    int source_node=0;
    //std::cout << "Enter the source node (0-based index): ";
    //std::cin >> source_node;
    distances[source_node * num_nodes + source_node] = 0;

    auto start_sequential = std::chrono::steady_clock::now();


    gg(graph,num_nodes);
    dd(distances,num_nodes);
    nspdp_sequential(graph, distances, num_nodes);
    dd(distances,num_nodes);


    auto end_sequential = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed_seconds_sequential = end_sequential - start_sequential;
    std::cout << "Sequential NSPDP execution time: " << elapsed_seconds_sequential.count() << "s\n";

    int* graph_gpu;
    int* distances_gpu;
    cudaMalloc(&graph_gpu, num_nodes * num_nodes * sizeof(int));
    cudaMalloc(&distances_gpu, num_nodes * num_nodes * sizeof(int));

    cudaMemcpy(graph_gpu, graph.data(), num_nodes * num_nodes * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(distances_gpu, distances.data(), num_nodes * num_nodes * sizeof(int), cudaMemcpyHostToDevice);

    int block_size = 256;
    int num_blocks = (num_nodes + block_size - 1) / block_size;

    auto start_cuda = std::chrono::steady_clock::now();
    nspdp_kernel<<<num_blocks, block_size>>>(graph_gpu, distances_gpu, num_nodes);
    cudaDeviceSynchronize();
    auto end_cuda = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed_seconds_cuda = end_cuda - start_cuda;
    dd(distances,num_nodes);

    std::cout << "CUDA NSPDP execution time: " << elapsed_seconds_cuda.count() << "s\n";

    cudaMemcpy(distances.data(), distances_gpu, num_nodes * num_nodes * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(graph_gpu);
    cudaFree(distances_gpu);

    return 0;
}


0 4 10 5 8 
4 0 10 8 8 
1 1 0 5 8 
8 8 5 0 10 
2 9 2 10 0 
0 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
0 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
6 6 2147483647 2147483647 2147483647 
2147483647 3 2147483647 7 2147483647 
Sequential NSPDP execution time: 2.7101e-05s
0 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
2147483647 2147483647 2147483647 2147483647 2147483647 
6 6 2147483647 2147483647 2147483647 
2147483647 3 2147483647 7 2147483647 
CUDA NSPDP execution time: 0.000231991s

